# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
# Study data files
weather_file = "../output_data/Exported_weather_data.csv"

# Read the mouse data and the study results
weather_data = pd.read_csv(weather_file)
weather_data=weather_data.drop('Unnamed: 0', axis=1)
weather_data.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Bredasdorp,-34.53,20.04,15.00,54,61,3.60,ZA,1595777264
1,Broken Hill,-31.95,141.43,10.00,64,95,2.60,AU,1595777327
2,São Filipe,14.90,-24.50,25.48,86,100,4.26,CV,1595777331
3,Bluff,-46.60,168.33,3.89,83,25,1.34,NZ,1595777334
4,Mataura,-46.19,168.86,3.89,83,1,1.34,NZ,1595777337


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
gmaps.configure(api_key=g_key)
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"].astype(float)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
ideal=weather_data.loc[(weather_data['Max_Temp']<30)]
ideal=ideal.loc[(ideal['Max_Temp']>20)]
ideal=ideal.loc[(ideal['Wind_Speed']<5)]
ideal=ideal.loc[(ideal['Humidity']<50)]
ideal.count()


City          30
Lat           30
Lng           30
Max_Temp      30
Humidity      30
Cloudiness    30
Wind_Speed    30
Country       29
Date          30
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df=ideal
hotel_df["Hotel Name"]=""
hotel_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
42,Karasburg,-28.02,18.75,20.08,23,0,2.65,NaN,1595777464,
58,Yumen,40.28,97.20,27.85,17,68,3.67,CN,1595777521,
65,Chapada dos Guimarães,-15.46,-55.75,24.00,44,0,2.10,BR,1595777544,
82,Nikolskoye,59.70,30.79,21.00,46,20,3.00,RU,1595777604,
83,Karratha,-20.74,116.85,20.35,38,39,3.44,AU,1595777607,


In [15]:
hotels=[]
count=0
# hotel=row['Hotel Name']
for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
    

    
    target=(f'{lat} , {lng}')
    target_search = "Hotel"
    target_radius = 5000
    target_type = "hotel"

# set up a parameters dictionary
    params = {
        "location": target,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary

    response = requests.get(base_url, params=params)
    places_data=response.json()
    try:
        hotel_df.loc[index,'Hotel Name']=places_data["results"][0]["name"]
#         hotels.append(places_data["results"][0]["name"])
#         hotel.append(places_data["results"][0]["name"])
        count+=1
        print(f'Found hotel for location {count}')
        
    except:
        count+=1
        print(f'No hotel for location {count}')
# for i in range(0,len(hotel_df['City'])):
#     print(places_data["results"][i]["name"])
#     print(places_data["results"][i]["vicinity"])

Found hotel for location 1
No hotel for location 2
Found hotel for location 3
Found hotel for location 4
Found hotel for location 5
Found hotel for location 6
Found hotel for location 7
Found hotel for location 8
Found hotel for location 9
Found hotel for location 10
Found hotel for location 11
Found hotel for location 12
Found hotel for location 13
Found hotel for location 14
Found hotel for location 15
No hotel for location 16
Found hotel for location 17
Found hotel for location 18
Found hotel for location 19
Found hotel for location 20
Found hotel for location 21
Found hotel for location 22
Found hotel for location 23
Found hotel for location 24
Found hotel for location 25
Found hotel for location 26
No hotel for location 27
Found hotel for location 28
Found hotel for location 29
Found hotel for location 30


In [16]:
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
42,Karasburg,-28.02,18.75,20.08,23,0,2.65,NaN,1595777464,U&K Bed & Breakfast
58,Yumen,40.28,97.20,27.85,17,68,3.67,CN,1595777521,
65,Chapada dos Guimarães,-15.46,-55.75,24.00,44,0,2.10,BR,1595777544,Pousada Penhasco
82,Nikolskoye,59.70,30.79,21.00,46,20,3.00,RU,1595777604,Tourist House - Sablino
83,Karratha,-20.74,116.85,20.35,38,39,3.44,AU,1595777607,Karratha International Hotel
123,Montepuez,-13.13,39.00,21.44,47,76,2.31,MZ,1595777746,RESIDÊNCIAL LANCHONETE
139,Saint George,37.10,-113.58,28.89,21,1,1.08,US,1595777797,Best Western Plus Abbey Inn
170,Arrondissement de Melun,48.50,2.75,26.67,41,28,4.10,FR,1595777901,Chateau De Rouillon
182,Lanzhou,36.06,103.79,24.44,47,0,1.61,CN,1595777942,Holiday Inn Hotel And Suites Lanzhou Center
188,Lavumisa,-27.31,31.89,27.00,28,0,1.00,SZ,1595777961,Royal Jozini Big 6 Private Estate


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))